In [1]:
import numpy as np
import pandas as pd
import re
import csv
from faker import Factory
from dateutil import parser
from collections import defaultdict
import datetime

In [2]:
# baca data review
review_df = pd.read_csv('all_review_skincare_female_daily.csv')
review_df.head(5)

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now
0,Ruthdayu,https://account.femaledaily.com/user/fd/Ruthdayu,5.0,Yes,10 Jan 2021,must have this product!👍😘 ini sheet mask dri A...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:26.288074
1,expectopatronum,https://account.femaledaily.com/user/fd/expect...,5.0,Yes,17 Dec 2020,Cuteee banget packaging plus nama2nya (Berry-M...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:29.476548
2,hilyaanafisa_,https://account.femaledaily.com/user/fd/hilyaa...,5.0,Yes,10 Jul 2020,dapet karena menang giveaway altheaxtroveskin ...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:32.623133
3,Syifareihany,https://account.femaledaily.com/user/fd/Syifar...,5.0,Yes,15 Mar 2022,first time aku pake masker huppies ini dari ka...,https://reviews.femaledaily.com/products/mask/...,A by BOM,Huppies,2023-10-05 17:43:44.423577
4,scalava,https://account.femaledaily.com/user/fd/scalava,2.0,No,10 Jun 2021,aku coba masker ini utk pertama kali. harga ny...,https://reviews.femaledaily.com/products/mask/...,A by BOM,Huppies,2023-10-05 17:43:50.620006


In [3]:
# melihat jumlah data
review_df.shape

(48015, 10)

In [4]:
# melihat tipe data tiap kolom
review_df.dtypes

user_name         object
user_link         object
star_rating      float64
is_recommend      object
review_date       object
user_review       object
review_link       object
product_brand     object
product_name      object
date_now          object
dtype: object

In [5]:
# mengganti nan dengan string kosong
review_df = review_df.replace('', np.nan)

In [6]:
# melihat jumlah data null
review_df.isnull().sum()

user_name           0
user_link           0
star_rating         0
is_recommend     5299
review_date         0
user_review         0
review_link         0
product_brand       0
product_name        0
date_now            0
dtype: int64

In [7]:
# melihat jumlah data duplikat berdasarkan kolom user_review dan user_name serta review_link
review_df.duplicated(subset=['user_review', 'user_name', 'review_link']).sum()

23608

In [8]:
# lihat data yang duplikat berdasarkan user_review dan user_name serta review_link
review_df[review_df.duplicated(['user_review', 'user_name', 'review_link'], keep=False)].sort_values(by=['user_review', 'user_name', 'review_link'])

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now
120,kirannisa,https://account.femaledaily.com/user/fd/kirannisa,5.0,NaN,06 Apr 2018,sejak pertama kali teracun seorang teman dan ...,https://reviews.femaledaily.com/products/treat...,A-True,Real Black Tea True Active Essence,2023-10-05 17:59:11.345206
23354,kirannisa,https://account.femaledaily.com/user/fd/kirannisa,5.0,NaN,06 Apr 2018,sejak pertama kali teracun seorang teman dan ...,https://reviews.femaledaily.com/products/treat...,A-True,Real Black Tea True Active Essence,2023-10-05 17:59:11.345206
19900,AdellaMajesty,https://account.femaledaily.com/user/fd/Adella...,2.0,No,26 Mar 2021,"!BREAK OUT! Asli jujur aku bingung banget, aku...",https://reviews.femaledaily.com/products/treat...,AXIS-Y,Artichoke Intensive Skin Barrier Ampoule,2023-10-07 09:30:47.638714
43134,AdellaMajesty,https://account.femaledaily.com/user/fd/Adella...,2.0,No,26 Mar 2021,"!BREAK OUT! Asli jujur aku bingung banget, aku...",https://reviews.femaledaily.com/products/treat...,AXIS-Y,Artichoke Intensive Skin Barrier Ampoule,2023-10-07 09:30:47.638714
8555,Syahnidaareo,https://account.femaledaily.com/user/fd/Syahni...,5.0,Yes,17 Sep 2020,#HonestReview :\r\n💦Avoskin Miraculous Refinin...,https://reviews.femaledaily.com/products/treat...,AVOSKIN,Miraculous Refining Serum,2023-10-06 19:55:51.997245
...,...,...,...,...,...,...,...,...,...,...
31368,neniyuniar,https://account.femaledaily.com/user/fd/neniyu...,5.0,Yes,24 Jul 2021,🧡 Kemasan dan Tekstur : Packaging Avoskin Mira...,https://reviews.femaledaily.com/products/treat...,AVOSKIN,Miraculous Refining Serum,2023-10-06 19:28:01.425223
2144,Beaxaud0n_,https://account.femaledaily.com/user/fd/Beaxau...,5.0,Yes,16 Feb 2023,🪄Holy grail🪄 Tekstur nya cair so easy to use &...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:33:24.612766
25378,Beaxaud0n_,https://account.femaledaily.com/user/fd/Beaxau...,5.0,Yes,16 Feb 2023,🪄Holy grail🪄 Tekstur nya cair so easy to use &...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:33:24.612766
20887,reginaaaaa99,https://account.femaledaily.com/user/fd/regina...,5.0,Yes,14 Jan 2022,🪴 Abib Heartleaf calming toner 🪴\r\n\r\n🍀 Ingr...,https://reviews.femaledaily.com/products/clean...,Abib Cosmetics,Heartleaf Calming Toner Skin Booster,2023-10-07 10:45:37.199698


In [9]:
# hapus data duplikat berdasarkan user_review dan user_name serta review_link dengan mempertahankan baris pertama
review_df.drop_duplicates(subset=['user_review', 'user_name', 'review_link'], keep='first', inplace=True)

In [10]:
# melihat jumlah data duplikat berdasarkan kolom user_review dan user_name serta review_link
review_df.duplicated().sum()

0

In [11]:
review_df.shape

(24407, 10)

## Convert Review Time Format

In [12]:
# melihat tanggal review contain 'ago'
review_df[review_df.review_date.str.lower().str.contains(' ago')]

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now
1548,dearelly,https://account.femaledaily.com/user/fd/dearelly,4.0,Yes,a day ago,"Pertama suka sama aromanya, kaya permen, segar...",https://reviews.femaledaily.com/products/mask/...,APIEU,Icing Sweet Bar Sheet Mask,2023-10-06 00:39:50.081199
1953,aifi,https://account.femaledaily.com/user/fd/aifi,3.0,Yes,an hour ago,Bisa dibilang cocok ngga cocok sih sama avoski...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:20:32.708983
1954,peyy,https://account.femaledaily.com/user/fd/peyy,5.0,Yes,a day ago,awal pake kerasa cekat cekit tapi udah pake la...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:20:35.816104
1955,Nurainiannasir,https://account.femaledaily.com/user/fd/Nurain...,5.0,Yes,a day ago,Pakai toner inii bener-bener bikin kulitku hal...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:20:39.011991
1956,kaseivallis,https://account.femaledaily.com/user/fd/kaseiv...,5.0,Yes,3 days ago,seminggu dua minggu pertama belom ngerasain ef...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:20:42.228605
1957,MomBee,https://account.femaledaily.com/user/fd/MomBee,5.0,Yes,5 days ago,nyari toner explo akhirnya ktemu ini stelah se...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:20:45.469364
1958,susimardian,https://account.femaledaily.com/user/fd/susima...,5.0,Yes,6 days ago,Aaa kenapa baru ketemu sekarang sihh🙈 belum ad...,https://reviews.femaledaily.com/products/clean...,AVOSKIN,Miraculous Refining Toner,2023-10-06 01:20:48.715385
7871,rimadwi21,https://account.femaledaily.com/user/fd/rimadwi21,5.0,Yes,5 days ago,serum kecintaan ku sepanjang masa❤️ dipake nya...,https://reviews.femaledaily.com/products/treat...,AVOSKIN,Miraculous Refining Serum,2023-10-06 19:10:51.851748
9128,Nurainiannasir,https://account.femaledaily.com/user/fd/Nurain...,5.0,Yes,2 days ago,Gaa usah di tanya lagiii ini mah.. harap maklu...,https://reviews.femaledaily.com/products/treat...,AVOSKIN,Miraculous Retinol Ampoule,2023-10-06 20:47:14.844546
9129,meliaafy,https://account.femaledaily.com/user/fd/meliaafy,5.0,Yes,3 days ago,produk retinol dari avoskin ini bagusss ukuran...,https://reviews.femaledaily.com/products/treat...,AVOSKIN,Miraculous Retinol Ampoule,2023-10-06 20:47:17.962205


In [13]:
# mengganti tanggal review yang contain 'ago' 
review_df['review_date_converted'] = review_df.review_date.str.replace("an hour", "1 hours")
review_df['review_date_converted'] = review_df.review_date_converted.str.replace("a day", "1 days")
review_df['review_date_converted'] = review_df.review_date_converted.str.replace("a few seconds", "1 minutes")

In [14]:
# convert tanggal review menjadi tanggal sekarang
convert_date = review_df['review_date_converted'].to_list()
review_df.date_now = pd.to_datetime(review_df.date_now)

In [15]:
date_convert = []
for index, time in enumerate(convert_date):
    if index < len(review_df):
        if time[-3:] == 'ago':
            parsed_time = [time.split()[:2]]
            time_dict = dict((format_time, float(amount)) for amount, format_time in parsed_time)
            time_delta = datetime.timedelta(**time_dict)
            past_time = review_df['date_now'].iloc[index] - time_delta
        else:
            past_time = parser.parse(time).date()
        date_convert.append(past_time)
    else:
        date_convert.append(None)  # Atau sesuaikan dengan nilai default yang sesuai

In [16]:
# menambahkan kolom baru berisi tanggal review yang sudah di convert
review_df['review_date_converted'] = date_convert

In [17]:
review_df.head(5)

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now,review_date_converted
0,Ruthdayu,https://account.femaledaily.com/user/fd/Ruthdayu,5.0,Yes,10 Jan 2021,must have this product!👍😘 ini sheet mask dri A...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:26.288074,2021-01-10
1,expectopatronum,https://account.femaledaily.com/user/fd/expect...,5.0,Yes,17 Dec 2020,Cuteee banget packaging plus nama2nya (Berry-M...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:29.476548,2020-12-17
2,hilyaanafisa_,https://account.femaledaily.com/user/fd/hilyaa...,5.0,Yes,10 Jul 2020,dapet karena menang giveaway altheaxtroveskin ...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:32.623133,2020-07-10
3,Syifareihany,https://account.femaledaily.com/user/fd/Syifar...,5.0,Yes,15 Mar 2022,first time aku pake masker huppies ini dari ka...,https://reviews.femaledaily.com/products/mask/...,A by BOM,Huppies,2023-10-05 17:43:44.423577,2022-03-15
4,scalava,https://account.femaledaily.com/user/fd/scalava,2.0,No,10 Jun 2021,aku coba masker ini utk pertama kali. harga ny...,https://reviews.femaledaily.com/products/mask/...,A by BOM,Huppies,2023-10-05 17:43:50.620006,2021-06-10


## impute is_recommend missing value

In [18]:
# melihat banyak data is_recommend
review_df.is_recommend.value_counts()

Yes    19273
No      2213
Name: is_recommend, dtype: int64

In [19]:
# melihat banyak data is_recommend yang null
review_df.is_recommend.isnull().sum()

2921

In [20]:
review_df.groupby(['star_rating', 'is_recommend'], dropna=False, as_index=False).size()

,star_rating,is_recommend,size
0,1.0,No,326
1,1.0,Yes,33
2,1.0,NaN,121
3,2.0,No,687
4,2.0,Yes,124
5,2.0,NaN,233
6,2.5,NaN,2
7,3.0,No,1014
8,3.0,Yes,1003
9,3.0,NaN,501


In [21]:
review_df['is_recommend_imputed'] = review_df.is_recommend.fillna('Unknown')

In [22]:
review_df[(review_df.is_recommend.str.lower().str.contains(r'no|yes', na=False))].sort_values(by=['review_date_converted'])

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now,review_date_converted,is_recommend_imputed
1206,lusikhairani,https://account.femaledaily.com/user/fd/lusikh...,3.0,Yes,18 Jun 2016,Awalnya memutuskan beli cleanser ini karena ke...,https://reviews.femaledaily.com/products/clean...,ANTIPODES,Hallelujah Lime and Patchouli Cleanser,2023-10-06 00:03:45.360601,2016-06-18 00:00:00.000000,Yes
7841,reedyanie,https://account.femaledaily.com/user/fd/reedyanie,5.0,Yes,06 Mar 2018,LEbih cocok pakai essence ini daripada pake s*...,https://reviews.femaledaily.com/products/treat...,AVOSKIN,Perfect Hydrating Treatment Essence,2023-10-06 19:05:54.664742,2018-03-06 00:00:00.000000,Yes
47435,arraqibtiah,https://account.femaledaily.com/user/fd/arraqi...,3.0,No,28 Apr 2018,"beli ini waktu sma, dulu ngerasa kl dia works ...",https://reviews.femaledaily.com/products/mask/...,Beautetox,Face and Body Mask,2023-11-27 00:28:30.178791,2018-04-28 00:00:00.000000,No
1089,beautytista,https://account.femaledaily.com/user/fd/beauty...,5.0,Yes,14 Jun 2018,Kulit saya ada banyak sekali bekas jerawat yan...,https://reviews.femaledaily.com/products/treat...,ANTIPODES,Apostle Skin-Brightening Serum,2023-10-05 23:45:44.495681,2018-06-14 00:00:00.000000,Yes
7783,mearvic,https://account.femaledaily.com/user/fd/mearvic,5.0,Yes,21 Jun 2018,"Ini serum TERMANTEB yang pernah gw coba, sebel...",https://reviews.femaledaily.com/products/treat...,AVOSKIN,Perfect Hydrating Treatment Essence,2023-10-06 18:56:27.877878,2018-06-21 00:00:00.000000,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
47014,_Ameliaefendi97,https://account.femaledaily.com/user/fd/_Ameli...,5.0,Yes,18 Nov 2023,Cleansing balm yang sangat membantu untuk memg...,https://reviews.femaledaily.com/products/makeu...,Barenbliss,K.O! Kombucha Deep Cleansing Balm,2023-11-26 23:41:22.803582,2023-11-18 00:00:00.000000,Yes
46977,Anggitakarima,https://account.femaledaily.com/user/fd/Anggit...,5.0,Yes,6 days ago,"LUCU BGT KEMASANNYA, GEMOYY!!! kirain isinya b...",https://reviews.femaledaily.com/products/moist...,Barenbliss,Lab Power! Oléomide Brightening Reviving Cream,2023-11-26 23:38:31.558819,2023-11-20 23:38:31.558819,Yes
46978,natashabule,https://account.femaledaily.com/user/fd/natash...,4.0,Yes,6 days ago,"Aku kira tutup kuningnya bisa dibuka, gataunya...",https://reviews.femaledaily.com/products/moist...,Barenbliss,Lab Power! Oléomide Brightening Reviving Cream,2023-11-26 23:38:34.754906,2023-11-20 23:38:34.754906,Yes
46780,Anggitakarima,https://account.femaledaily.com/user/fd/Anggit...,3.0,Yes,3 days ago,Awal pake ini aman banget tapi waktu cobain se...,https://reviews.femaledaily.com/products/treat...,Barenbliss,Double Shot! C Day A Night Deep Brightening Do...,2023-11-26 23:24:52.310278,2023-11-23 23:24:52.310278,Yes


In [23]:
review_df.sort_values(by=['review_date_converted']).head(1)

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now,review_date_converted,is_recommend_imputed
1890,santisiera,https://account.femaledaily.com/user/fd/santis...,4.5,NaN,26 Jul 2015,Serum ini berfungsi untuk menumbuhkan bulu mat...,https://reviews.femaledaily.com/products/treat...,APOTCARE,Optilash Lash Enhancing Serum,2023-10-06 01:11:55.202803,2015-07-26,Unknown


In [24]:
review_df

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now,review_date_converted,is_recommend_imputed
0,Ruthdayu,https://account.femaledaily.com/user/fd/Ruthdayu,5.0,Yes,10 Jan 2021,must have this product!👍😘 ini sheet mask dri A...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:26.288074,2021-01-10,Yes
1,expectopatronum,https://account.femaledaily.com/user/fd/expect...,5.0,Yes,17 Dec 2020,Cuteee banget packaging plus nama2nya (Berry-M...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:29.476548,2020-12-17,Yes
2,hilyaanafisa_,https://account.femaledaily.com/user/fd/hilyaa...,5.0,Yes,10 Jul 2020,dapet karena menang giveaway altheaxtroveskin ...,https://reviews.femaledaily.com/products/mask/...,A Bonne,Berry-Me-Baby Sheetmask,2023-10-05 17:43:32.623133,2020-07-10,Yes
3,Syifareihany,https://account.femaledaily.com/user/fd/Syifar...,5.0,Yes,15 Mar 2022,first time aku pake masker huppies ini dari ka...,https://reviews.femaledaily.com/products/mask/...,A by BOM,Huppies,2023-10-05 17:43:44.423577,2022-03-15,Yes
4,scalava,https://account.femaledaily.com/user/fd/scalava,2.0,No,10 Jun 2021,aku coba masker ini utk pertama kali. harga ny...,https://reviews.femaledaily.com/products/mask/...,A by BOM,Huppies,2023-10-05 17:43:50.620006,2021-06-10,No
...,...,...,...,...,...,...,...,...,...,...,...,...
48010,tari1107,https://account.femaledaily.com/user/fd/tari1107,5.0,NaN,22 Nov 2017,ini merupakan aloevera gel pertama yg aku pake...,https://reviews.femaledaily.com/products/moist...,Beauty K,Jeju Fresh Aloe,2023-11-27 01:49:00.956555,2017-11-22,Unknown
48011,ayulrst,https://account.femaledaily.com/user/fd/ayulrst,4.0,NaN,18 Nov 2017,Bingung sih mau blg apa soal product ini krn j...,https://reviews.femaledaily.com/products/moist...,Beauty K,Jeju Fresh Aloe,2023-11-27 01:49:10.253968,2017-11-18,Unknown
48012,nino0692,https://account.femaledaily.com/user/fd/nino0692,5.0,NaN,26 Oct 2017,Aloe Shooting Gel pertama yang aku coba. Sampa...,https://reviews.femaledaily.com/products/moist...,Beauty K,Jeju Fresh Aloe,2023-11-27 01:49:19.468083,2017-10-26,Unknown
48013,Dvnpricilia,https://account.femaledaily.com/user/fd/Dvnpri...,2.0,NaN,13 Oct 2017,"Excited bgt waktu liat ini di century, tnpa cu...",https://reviews.femaledaily.com/products/moist...,Beauty K,Jeju Fresh Aloe,2023-11-27 01:49:28.680314,2017-10-13,Unknown


In [25]:
# menghapus kolom yang tidak digunakan
review_df = review_df.drop(['user_link', 'is_recommend', 'review_date', 'user_review', 'review_link', 'date_now'],
                           axis=1)

In [26]:
review_df

,user_name,star_rating,product_brand,product_name,review_date_converted,is_recommend_imputed
0,Ruthdayu,5.0,A Bonne,Berry-Me-Baby Sheetmask,2021-01-10,Yes
1,expectopatronum,5.0,A Bonne,Berry-Me-Baby Sheetmask,2020-12-17,Yes
2,hilyaanafisa_,5.0,A Bonne,Berry-Me-Baby Sheetmask,2020-07-10,Yes
3,Syifareihany,5.0,A by BOM,Huppies,2022-03-15,Yes
4,scalava,2.0,A by BOM,Huppies,2021-06-10,No
...,...,...,...,...,...,...
48010,tari1107,5.0,Beauty K,Jeju Fresh Aloe,2017-11-22,Unknown
48011,ayulrst,4.0,Beauty K,Jeju Fresh Aloe,2017-11-18,Unknown
48012,nino0692,5.0,Beauty K,Jeju Fresh Aloe,2017-10-26,Unknown
48013,Dvnpricilia,2.0,Beauty K,Jeju Fresh Aloe,2017-10-13,Unknown


In [27]:
review_df[['user_name', 'product_brand', 'product_name']].duplicated().sum()

1190

In [28]:
review_df[review_df.duplicated(['user_name', 'product_brand', 'product_name'], keep=False)]

,user_name,star_rating,product_brand,product_name,review_date_converted,is_recommend_imputed
47,erlianiskndr,5.0,A-True,Real Black Tea True Active Essence,2023-03-21,Yes
48,ciitrasarii,5.0,A-True,Real Black Tea True Active Essence,2022-05-22,Yes
49,curatedbytika,5.0,A-True,Real Black Tea True Active Essence,2022-03-14,Unknown
50,Citraanna,5.0,A-True,Real Black Tea True Active Essence,2022-02-27,Yes
51,nantrip,4.0,A-True,Real Black Tea True Active Essence,2021-11-04,Yes
...,...,...,...,...,...,...
47849,amadeac,3.0,Beauty In The Pot,Masker Organik,2020-06-04,Yes
47850,medinasfryn,3.0,Beauty In The Pot,Masker Organik,2019-10-08,No
47853,nelyaulia,4.5,Beauty In The Pot,Masker Organik,2015-11-27,Unknown
47854,dunianyabunga,3.0,Beauty In The Pot,Masker Organik,2015-11-10,Unknown


karena dilihat dari tanggal merupakan review yang berbeda sehingga data duplikat ini tidak perlu dihapus karena kemungkinan dapat mempengaruhi dalam membuat model collaborative filtering 

In [29]:
# simpan file review yang sudah di cleaning
review_df.to_csv('review_clean_1_1.csv', index=False)

In [31]:
review_df.shape

(24407, 6)

# Batassssss

## Menambahkan id untuk product dan user

In [25]:
# baca data review yang sudah di cleaning
review_df = pd.read_csv('review_clean_1_1.csv')

In [26]:
review_df.nunique()

user_name                 9580
star_rating                  8
user_review              14539
product_brand               30
product_name               179
review_date_converted     2342
is_recommend_imputed         3
dtype: int64

In [27]:
# Check product_name of ratings
review_df.groupby("product_name")['star_rating'].count().sort_values(ascending=False).head()

product_name
Your Skin Bae Serum                    2639
Miraculous Refining Toner              2100
Perfect Hydrating Treatment Essence    1380
Hydrating Treatment Essence             779
Intensive Nourishing Eye Cream          755
Name: star_rating, dtype: int64

In [28]:
review_df.groupby("product_name")['star_rating'].count().sort_values(ascending=True).head()

product_name
Vitamin B Serum with Niacinamide + Panthenol    1
LIVELY SuperBarrier™ Cica-Panthenol Serum       1
Joyous Protein-Rich Night Replenish Serum       1
Aqua Boosting Essence Toner                     1
Irido Radiant Daily Serum                       1
Name: star_rating, dtype: int64

In [29]:
# Impor library pandas
import pandas as pd

# Buat peta pemetaan (mapping) dari product_name ke ID numerik
unique_product_names = review_df['product_name'].unique()
product_id_mapping = {product_name: id for id, product_name in enumerate(unique_product_names)}

# Tambahkan kolom 'product_id' ke DataFrame dengan ID numerik
review_df['product_id'] = review_df['product_name'].map(product_id_mapping)
review_df

,user_name,star_rating,user_review,product_brand,product_name,review_date_converted,is_recommend_imputed,product_id
0,Ruthdayu,5.0,must have this product!👍😘 ini sheet mask dri A...,A Bonne,Berry-Me-Baby Sheetmask,2021-01-10 00:00:00.000000,Yes,0
1,expectopatronum,5.0,Cuteee banget packaging plus nama2nya (Berry-M...,A Bonne,Berry-Me-Baby Sheetmask,2020-12-17 00:00:00.000000,Yes,0
2,hilyaanafisa_,5.0,dapet karena menang giveaway altheaxtroveskin ...,A Bonne,Berry-Me-Baby Sheetmask,2020-07-10 00:00:00.000000,Yes,0
3,Syifareihany,5.0,first time aku pake masker huppies ini dari ka...,A by BOM,Huppies,2022-03-15 00:00:00.000000,Yes,1
4,scalava,2.0,aku coba masker ini utk pertama kali. harga ny...,A by BOM,Huppies,2021-06-10 00:00:00.000000,No,1
...,...,...,...,...,...,...,...,...
14602,AlyaMaharani,5.0,serius recommended banget dah. kulitku normal ...,Acnes,Derma Care Gentle Cleanser,2021-02-24,Yes,178
14603,cseptilia,5.0,Mungkin aku terlalu cepat buat kasih reviews p...,Acnes,Derma Care Gentle Cleanser,2021-02-23,Yes,178
14604,RosmaCindy,5.0,Penasaran sama produk ini. karena aku udah cob...,Acnes,Derma Care Gentle Cleanser,2021-02-16,Yes,178
14605,Drupadi,5.0,Salah satu produk acnes dermacare yang aku pak...,Acnes,Derma Care Gentle Cleanser,2021-02-13,Yes,178


In [30]:
product_name_to_find = "Your Skin Bae Serum"
product_id = review_df.loc[review_df['product_name'] == product_name_to_find, 'product_id'].iloc[0]
print(f"ID numerik untuk '{product_name_to_find}' adalah {product_id}")

ID numerik untuk 'Your Skin Bae Serum' adalah 115


In [31]:
# Check user_name of ratings
review_df.groupby("user_name")['star_rating'].count().sort_values(ascending=False).head()

user_name
popy_mercylin    16
tantihira        15
Clarentia01_     14
lavenda          14
lanaarahma       12
Name: star_rating, dtype: int64

In [32]:
# Check user_name of ratings
review_df.groupby("user_name")['star_rating'].count().sort_values(ascending=True).head()

user_name
01lely                     1
kathlynputri               1
katarinachristinnatalia    1
kaseivallis                1
karyngabriella             1
Name: star_rating, dtype: int64

In [33]:
# Impor library pandas
import pandas as pd

# Buat peta pemetaan (mapping) dari user_name ke ID numerik
unique_product_names = review_df['user_name'].unique()
product_id_mapping = {product_name: id for id, product_name in enumerate(unique_product_names)}

# Tambahkan kolom 'user_id' ke DataFrame dengan ID numerik
review_df['user_id'] = review_df['user_name'].map(product_id_mapping)
review_df

,user_name,star_rating,user_review,product_brand,product_name,review_date_converted,is_recommend_imputed,product_id,user_id
0,Ruthdayu,5.0,must have this product!👍😘 ini sheet mask dri A...,A Bonne,Berry-Me-Baby Sheetmask,2021-01-10 00:00:00.000000,Yes,0,0
1,expectopatronum,5.0,Cuteee banget packaging plus nama2nya (Berry-M...,A Bonne,Berry-Me-Baby Sheetmask,2020-12-17 00:00:00.000000,Yes,0,1
2,hilyaanafisa_,5.0,dapet karena menang giveaway altheaxtroveskin ...,A Bonne,Berry-Me-Baby Sheetmask,2020-07-10 00:00:00.000000,Yes,0,2
3,Syifareihany,5.0,first time aku pake masker huppies ini dari ka...,A by BOM,Huppies,2022-03-15 00:00:00.000000,Yes,1,3
4,scalava,2.0,aku coba masker ini utk pertama kali. harga ny...,A by BOM,Huppies,2021-06-10 00:00:00.000000,No,1,4
...,...,...,...,...,...,...,...,...,...
14602,AlyaMaharani,5.0,serius recommended banget dah. kulitku normal ...,Acnes,Derma Care Gentle Cleanser,2021-02-24,Yes,178,9577
14603,cseptilia,5.0,Mungkin aku terlalu cepat buat kasih reviews p...,Acnes,Derma Care Gentle Cleanser,2021-02-23,Yes,178,9349
14604,RosmaCindy,5.0,Penasaran sama produk ini. karena aku udah cob...,Acnes,Derma Care Gentle Cleanser,2021-02-16,Yes,178,8130
14605,Drupadi,5.0,Salah satu produk acnes dermacare yang aku pak...,Acnes,Derma Care Gentle Cleanser,2021-02-13,Yes,178,9578


In [34]:
user_name_to_find = "popy_mercylin"
user_id = review_df.loc[review_df['user_name'] == user_name_to_find, 'user_id'].iloc[0]
print(f"ID numerik untuk '{user_name_to_find}' adalah {user_id}")

ID numerik untuk 'popy_mercylin' adalah 87


In [35]:
review_df

,user_name,star_rating,user_review,product_brand,product_name,review_date_converted,is_recommend_imputed,product_id,user_id
0,Ruthdayu,5.0,must have this product!👍😘 ini sheet mask dri A...,A Bonne,Berry-Me-Baby Sheetmask,2021-01-10 00:00:00.000000,Yes,0,0
1,expectopatronum,5.0,Cuteee banget packaging plus nama2nya (Berry-M...,A Bonne,Berry-Me-Baby Sheetmask,2020-12-17 00:00:00.000000,Yes,0,1
2,hilyaanafisa_,5.0,dapet karena menang giveaway altheaxtroveskin ...,A Bonne,Berry-Me-Baby Sheetmask,2020-07-10 00:00:00.000000,Yes,0,2
3,Syifareihany,5.0,first time aku pake masker huppies ini dari ka...,A by BOM,Huppies,2022-03-15 00:00:00.000000,Yes,1,3
4,scalava,2.0,aku coba masker ini utk pertama kali. harga ny...,A by BOM,Huppies,2021-06-10 00:00:00.000000,No,1,4
...,...,...,...,...,...,...,...,...,...
14602,AlyaMaharani,5.0,serius recommended banget dah. kulitku normal ...,Acnes,Derma Care Gentle Cleanser,2021-02-24,Yes,178,9577
14603,cseptilia,5.0,Mungkin aku terlalu cepat buat kasih reviews p...,Acnes,Derma Care Gentle Cleanser,2021-02-23,Yes,178,9349
14604,RosmaCindy,5.0,Penasaran sama produk ini. karena aku udah cob...,Acnes,Derma Care Gentle Cleanser,2021-02-16,Yes,178,8130
14605,Drupadi,5.0,Salah satu produk acnes dermacare yang aku pak...,Acnes,Derma Care Gentle Cleanser,2021-02-13,Yes,178,9578


In [36]:
review_df.drop(columns=['user_name',], inplace=True)

In [37]:
review_df

,star_rating,user_review,product_brand,product_name,review_date_converted,is_recommend_imputed,product_id,user_id
0,5.0,must have this product!👍😘 ini sheet mask dri A...,A Bonne,Berry-Me-Baby Sheetmask,2021-01-10 00:00:00.000000,Yes,0,0
1,5.0,Cuteee banget packaging plus nama2nya (Berry-M...,A Bonne,Berry-Me-Baby Sheetmask,2020-12-17 00:00:00.000000,Yes,0,1
2,5.0,dapet karena menang giveaway altheaxtroveskin ...,A Bonne,Berry-Me-Baby Sheetmask,2020-07-10 00:00:00.000000,Yes,0,2
3,5.0,first time aku pake masker huppies ini dari ka...,A by BOM,Huppies,2022-03-15 00:00:00.000000,Yes,1,3
4,2.0,aku coba masker ini utk pertama kali. harga ny...,A by BOM,Huppies,2021-06-10 00:00:00.000000,No,1,4
...,...,...,...,...,...,...,...,...
14602,5.0,serius recommended banget dah. kulitku normal ...,Acnes,Derma Care Gentle Cleanser,2021-02-24,Yes,178,9577
14603,5.0,Mungkin aku terlalu cepat buat kasih reviews p...,Acnes,Derma Care Gentle Cleanser,2021-02-23,Yes,178,9349
14604,5.0,Penasaran sama produk ini. karena aku udah cob...,Acnes,Derma Care Gentle Cleanser,2021-02-16,Yes,178,8130
14605,5.0,Salah satu produk acnes dermacare yang aku pak...,Acnes,Derma Care Gentle Cleanser,2021-02-13,Yes,178,9578


In [38]:
# simpan file review ke csv
review_df.to_csv('review_id_1_1.csv', index=False)